In [2]:
# os is used for Ilka's linux :)
#import os
#os.environ["QT_QPA_PLATFORM"] = "xcb"

#To activate Elena's Environment conda activate D:/Elena/Cuarto/VC/Enviroments

from ultralytics import YOLO
import cv2
import pytesseract
import pandas as pd

train

In [3]:
# Load model
# model = YOLO("yolo11n.yaml")  # build a new model from YAML
# model = YOLO("yolo11n.pt")  # load a pretrained model (recommended for training)
model = YOLO("yolo11n.yaml").load("yolo11n.pt")  # build from YAML and transfer weights

# Train model
#results = model.train(data="./datasets/selfmadeYOLO/dataset.yaml", epochs=50, imgsz=640, batch=4)

Transferred 499/499 items from pretrained weights


predict

In [4]:
model_objects = YOLO("yolo11n.pt")
model_lipl = YOLO("datasets/selfmadeYOLO/runs/detect/50-epochs/weights/best.pt")
video_path = './test-vid-prof/test-vid-prof.mp4'  
output_path = './test-vid-prof/test-vid-prof-count.mp4'
csv_output_path = "./test-tracking/test-vid-prof-count.csv" 
out_csv = []
class_counts = {}
apply_blurring = False

# all class names for the object model
classNames = [
    "person", "bicycle", "car", "motorbike", "aeroplane", "bus", "train", "truck", "boat",
    "traffic light", "fire hydrant", "stop sign", "parking meter", "bench", "bird", "cat",
    "dog", "horse", "sheep", "cow", "elephant", "bear", "zebra", "giraffe", "backpack", "umbrella",
    "handbag", "tie", "suitcase", "frisbee", "skis", "snowboard", "sports ball", "kite", "baseball bat",
    "baseball glove", "skateboard", "surfboard", "tennis racket", "bottle", "wine glass", "cup",
    "fork", "knife", "spoon", "bowl", "banana", "apple", "sandwich", "orange", "broccoli",
    "carrot", "hot dog", "pizza", "donut", "cake", "chair", "sofa", "pottedplant", "bed",
    "diningtable", "toilet", "tvmonitor", "laptop", "mouse", "remote", "keyboard", "cell phone",
    "microwave", "oven", "toaster", "sink", "refrigerator", "book", "clock", "vase", "scissors",
    "teddy bear", "hair drier", "toothbrush"
]

tbd_classes = [
    "person", "bicycle", "car", "motorbike", "aeroplane", "bus", "train", "truck", "boat", "license plate"
]

In [4]:
# function for writing the csv

def writeToCsv(frame: int, cls: str, conf: float, trId: int, 
               x1: int, y1: int, x2:int, y2:int, 
               lipl: bool=False, mx1: int=0, my1: int=0, mx2: int=0, my2: int=0, text: str=""):
        out_csv.append({
            "frame": frame,
            "object_type": cls,
            "confidence": conf,
            "tracking_id": trId,
            "x1": x1,
            "y1": y1,
            "x2": x2,
            "y2": y2,
            "license_plate": lipl,
            "mx1": mx1,  # Placeholder for your additional coordinates if needed
            "my1": my1,
            "mx2": mx2,
            "my2": my2,
            "license_plate_text": text
    })
        
def hasTrackId(boxId) -> int:
    if boxId is not None:
        return boxId.int().cpu().item()
    else:
        return -1

predict with tracking, blurring people and license plates

In [8]:
out_csv = []

vid = cv2.VideoCapture(video_path)

# video configurations
fps = int(vid.get(cv2.CAP_PROP_FPS))
width = int(vid.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(vid.get(cv2.CAP_PROP_FRAME_HEIGHT))
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out_vid = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

while True:
    ret, img = vid.read()
    frame_no = vid.get(cv2.CAP_PROP_POS_FRAMES)
    print()
    print("processing frame number ", frame_no)

    if not ret:
        break

    # apply tracking
    res_obj = model_objects.track(img)
    res_lipl = model_lipl.track(img)

    # annotate frame
    img = res_obj[0].plot() 
    img = res_lipl[0].plot(img=img)

    # apply object detection
    for r in res_obj:
        boxes = r.boxes

        for box in boxes:
            # get object coordinates
            x1, y1, x2, y2 = map(int, box.xyxy[0])  # Convertir a enteros

            # get detected class
            cls = int(box.cls[0])
            detected_class = classNames[cls]

            # apply face blurring
            if apply_blurring and detected_class == "person":
                # blurr the upper third of a person -> only the head
                face_region = img[y1:y1 + (y2 - y1) // 3, x1:x2]
                blurred_face = cv2.GaussianBlur(face_region, (51, 51), 30)
                img[y1:y1 + (y2 - y1) // 3, x1:x2] = blurred_face

            track_id = hasTrackId(box.id)
            
            if detected_class in tbd_classes:
                writeToCsv(frame_no, detected_class, box.conf.item(), track_id, x1, y1, x2, y2)

            if detected_class in class_counts:
                class_counts[detected_class] += 1
            else:
                class_counts[detected_class] = 1

    # apply license plate model
    for r in res_lipl:
        boxes = r.boxes

        for box in boxes:
            # get object coordinates
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            detected_class = "license plate"

            # apply blurring
            if apply_blurring:
                plate_region = img[y1:y2, x1:x2]
                blurred_plate = cv2.GaussianBlur(plate_region, (51, 51), 30)
                img[y1:y2, x1:x2] = blurred_plate 

            # detect text
            lipl_crop = img[y1:y2, x1:x2]
            gray_crop = cv2.cvtColor(lipl_crop, cv2.COLOR_BGR2GRAY)
            text = pytesseract.image_to_string(gray_crop, config='--psm 7').strip()  # PSM 7 for single-line text
            #print("Detected License Plate Number:", text)
            
            # test if the detected license plate number contains a letter or a digit
            if(not any(char.isdigit() or char.isalpha() for char in text)):
                text = "not detected"
            
            track_id = hasTrackId(box.id)
            writeToCsv(frame_no, detected_class, box.conf.item(), track_id, x1, y1, x2, y2, True, x1, y1, x2, y2, text.strip())

            if detected_class in class_counts:
                class_counts[detected_class] += 1
            else:
                class_counts[detected_class] = 1

    # write frame to output video
    out_vid.write(img)

    # Optional: show video
    cv2.imshow('blurring', img)

    # end with ESC
    if cv2.waitKey(20) == 27:
        break

# free resources
vid.release()
out_vid.release()
cv2.destroyAllWindows()

# write csv in file
print()
print("Saving data to csv...")
df = pd.DataFrame(out_csv)
df.to_csv(csv_output_path, index=False)
print("Done!")
print()
print("Here are the class counts:")
print(class_counts)



processing frame number  1.0

0: 384x640 1 car, 1 bus, 77.8ms
Speed: 2.9ms preprocess, 77.8ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 87.9ms
Speed: 3.2ms preprocess, 87.9ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

processing frame number  2.0

0: 384x640 1 car, 1 bus, 155.8ms
Speed: 15.4ms preprocess, 155.8ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 license plate, 72.7ms
Speed: 2.9ms preprocess, 72.7ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)

processing frame number  3.0

0: 384x640 1 car, 1 bus, 164.7ms
Speed: 5.1ms preprocess, 164.7ms inference, 2.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 license plate, 71.5ms
Speed: 4.5ms preprocess, 71.5ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)

processing frame number  4.0

0: 384x640 1 car, 1 bus, 181.7ms
Speed: 4.7ms preprocess, 181.7ms inference, 2.7ms po

detection and counting of people and cars


0: 384x640 4 cars, 1 bus, 89.8ms
Speed: 4.9ms preprocess, 89.8ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 cars, 1 bus, 104.6ms
Speed: 16.2ms preprocess, 104.6ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 5 cars, 1 bus, 114.1ms
Speed: 3.3ms preprocess, 114.1ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 cars, 1 bus, 91.2ms
Speed: 7.0ms preprocess, 91.2ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 cars, 1 bus, 258.5ms
Speed: 10.0ms preprocess, 258.5ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 persons, 3 cars, 1 bus, 99.5ms
Speed: 5.0ms preprocess, 99.5ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 cars, 1 bus, 87.8ms
Speed: 3.1ms preprocess, 87.8ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 2 cars, 1 bus, 93.2ms
Speed: 9.0